In [1]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 3000, clientId=11) # kavi tws live
# ib = IB().connect('127.0.0.1', 3000, clientId=11) # kavi IBG live

In [10]:
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math

#...assignments

m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.05       # Upper or Lower base multiple for prices

min_rom = 0.8
min_pop = 0.98

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

#...get current positions
#________________________

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
df_p['dte'] = [(util.parseIBDatetime(d.lastTradeDateOrContractMonth) - datetime.datetime.now().date()).days for d in df_p.contract]

discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]

# ignore data for dte < 3 days. These are as good as gone.
df_h = df_p[df_p.dte > 3].reset_index(drop=True)

hqc = ib.qualifyContracts(*df_h.contract)
df_h = df_h.assign(qual_contract=hqc)

# ...sowing prepration
#_____________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

#...build the high-pop-roc dataframe
fs = listdir('./zdata/')

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

df_opt = pd.concat([pd.read_pickle('./zdata/'+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle('./zdata/'+f).reset_index() for f in ohlcs], axis=0, sort=True)

df_und = pd.concat([pd.read_pickle('./zdata/'+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

In [11]:
# take only high rom and pops
df_opt1 = df_opt[(df_opt.rom > min_rom) & 
                   (df_opt.dte > 2) &
                   (df_opt['pop'] > min_pop)]

df_mlot = df_und[['symbol', 'lot', 'marginpct']]

df_opt2 = df_opt1.merge(df_mlot, on='symbol', how='inner')

df_opt2['margin'] = df_opt2.marginpct * df_opt2.lot * df_opt2.undPrice

df_opt2['expPrice'] = round(round((df_opt2.price+0.1)/ base) * base, -int(math.floor(math.log10(base))))

# WARNING!!!... Make df the dataframe that you want to execute on!
df = df_opt2.copy()   # make this the last dataframe to get the orders placed

contracts = [c for c in df_opt2.option]

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]

print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving INR {:,.0f}'.format(len(contracts), 
                                                                                                                     len(df.symbol.unique()), sum(df.margin), av_funds*max_nlvp, sum(df.expPrice*df.lot)))

51 contracts from 17 scrips, consuming 5,361,315 margin from full available funds of 13,856,169, giving INR 999,598


In [12]:
df

,bsmPrice,dte,expiry,option,pop,price,right,rom,stdev,strike,symbol,undPrice,volatility,lot,marginpct,margin,expPrice
0,0.00,28,20190228,"Option(conId=343994856, symbol='IFCI', lastTra...",0.995309,1.45,P,5.436594,0.566032,11.0,IFCI,13.60,0.306900,35000,0.1765,84014.00000,1.55
1,0.00,27,20190228,"Option(conId=343978594, symbol='RPOWER', lastT...",1.000000,1.85,P,3.411373,1.593084,17.0,RPOWER,26.50,0.323467,16000,0.1910,80984.00000,1.95
2,0.00,27,20190228,"Option(conId=343978757, symbol='RPOWER', lastT...",0.999968,1.00,C,1.843986,1.593084,38.0,RPOWER,26.50,0.323467,16000,0.1910,80984.00000,1.10
3,0.00,28,20190228,"Option(conId=343967616, symbol='DISHTV', lastT...",0.997551,3.10,C,2.063502,2.763005,31.0,DISHTV,23.65,0.339295,8000,0.5717,108165.64000,3.20
4,0.01,56,20190328,"Option(conId=347845785, symbol='DISHTV', lastT...",0.987848,4.80,C,1.597550,3.291550,33.0,DISHTV,23.65,0.362158,8000,0.5717,108165.64000,4.90
5,0.01,56,20190328,"Option(conId=347845792, symbol='DISHTV', lastT...",0.993106,4.60,C,1.530985,3.291550,34.0,DISHTV,23.65,0.362158,8000,0.5717,108165.64000,4.70
6,0.00,56,20190328,"Option(conId=347845802, symbol='DISHTV', lastT...",0.996174,4.40,C,1.464421,3.291550,35.0,DISHTV,23.65,0.362158,8000,0.5717,108165.64000,4.50
7,0.00,56,20190328,"Option(conId=347845809, symbol='DISHTV', lastT...",0.997919,4.20,C,1.397856,3.291550,36.0,DISHTV,23.65,0.362158,8000,0.5717,108165.64000,4.30
8,0.00,56,20190328,"Option(conId=347845819, symbol='DISHTV', lastT...",0.998889,4.00,C,1.331292,3.291550,37.0,DISHTV,23.65,0.362158,8000,0.5717,108165.64000,4.10
9,0.00,56,20190328,"Option(conId=347845827, symbol='DISHTV', lastT...",0.999416,3.85,C,1.281368,3.291550,38.0,DISHTV,23.65,0.362158,8000,0.5717,108165.64000,3.95


In [ ]:
cols = ['right', 'symbol', 'strike', 'dte', 'undPrice', 'pop', 'rom', 'price', 'option', 'expPrice']
df[cols].sort_values(['right', 'symbol', 'strike'])

In [9]:
# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchlist = [('DES', s, 'STK', 'NSE') for s in df.symbol.unique()]
df_watch = util.df(watchlist)

df_watch.to_csv('./zdata/watchlist.csv', index=None, header=False)